# Multi Layer Perceptron

In [97]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score

In [98]:
data=pd.read_csv("/content/data1.csv")
data.head()

,x_1,x_2,y
0,1.70600,2.8859,1
1,-0.70611,3.5617,-1
2,3.42530,1.9469,1
3,2.53570,1.5779,1
4,1.68960,3.4147,-1


In [99]:
data.describe()

,x_1,x_2,y
count,400.000000,400.000000,400.000000
mean,2.631745,2.657470,-0.020000
std,1.030100,1.002468,1.001052
min,-0.706110,-0.460070,-1.000000
25%,1.955625,1.963600,-1.000000
50%,2.657900,2.669150,-1.000000
75%,3.268950,3.339475,1.000000
max,6.314200,5.309100,1.000000


In [100]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x_1     400 non-null    float64
 1   x_2     400 non-null    float64
 2   y       400 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 9.5 KB


In [101]:
data_path="/content/data1.csv"
data_np=np.loadtxt(data_path, delimiter=",", skiprows=1,dtype=np.float32)

data_np

array([[ 1.706  ,  2.8859 ,  1.     ],
       [-0.70611,  3.5617 , -1.     ],
       [ 3.4253 ,  1.9469 ,  1.     ],
       ...,
       [ 3.0927 ,  4.2169 , -1.     ],
       [ 3.81   ,  3.2415 ,  1.     ],
       [ 4.7261 ,  2.8226 ,  1.     ]], dtype=float32)

In [102]:
data_t=torch.from_numpy(data_np)
data_t.shape

torch.Size([400, 3])

In [103]:
400*(80/100)

320.0

In [104]:
x, y = data_t[:,:-1],data_t[:,-1].long()
x.shape,y.shape

(torch.Size([400, 2]), torch.Size([400]))

In [105]:
y[y==-1]=0

In [106]:
x_train, y_train, x_test, y_test = x[:320], y[:320], x[320:], y[320:] #80% training data
x_train.shape, y_train.shape, x_test.shape, y_test.shape

(torch.Size([320, 2]),
 torch.Size([320]),
 torch.Size([80, 2]),
 torch.Size([80]))

In [107]:
model=nn.Sequential(
    nn.Linear(2,32),
    nn.ReLU(),
    nn.Linear(32,64),
    nn.ReLU(),
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32,2),
)

loss_fn=nn.CrossEntropyLoss()

def training_loop(n_epochs, loss_fn, x_train, y_train, model, optimizer):
  for i in range(n_epochs):
    loss=0
    for j in range(len(x_train)):
      y_pred=model(x_train[j].unsqueeze(0))
      train_loss=loss_fn(y_pred,y_train[j].unsqueeze(0))

      optimizer.zero_grad()
      train_loss.backward()
      optimizer.step()

      loss+=train_loss.item()

    print(f"Epoch: {i}, Train_loss: {loss/len(x_train)}")
  return [i for i in model.parameters()]


In [108]:
x_train.shape

torch.Size([320, 2])

In [109]:
model_params = training_loop(
                    n_epochs=110,
                    loss_fn=loss_fn,
                    x_train=x_train,
                    y_train=y_train,
                    model=model,
                    optimizer=optim.SGD(model.parameters(),lr=1e-2)
                )

Epoch: 0, Train_loss: 0.6100366719067096
Epoch: 1, Train_loss: 0.40648643012245883
Epoch: 2, Train_loss: 0.3866714673091337
Epoch: 3, Train_loss: 0.37098899509983313
Epoch: 4, Train_loss: 0.35599159293306004
Epoch: 5, Train_loss: 0.35111676579581397
Epoch: 6, Train_loss: 0.34629980138197425
Epoch: 7, Train_loss: 0.34255325801968867
Epoch: 8, Train_loss: 0.3380414615094196
Epoch: 9, Train_loss: 0.3345677886994963
Epoch: 10, Train_loss: 0.3314978048714693
Epoch: 11, Train_loss: 0.32749009137478424
Epoch: 12, Train_loss: 0.3238869398817769
Epoch: 13, Train_loss: 0.3199807891665841
Epoch: 14, Train_loss: 0.3170540118138888
Epoch: 15, Train_loss: 0.31383957701182225
Epoch: 16, Train_loss: 0.3099611275494681
Epoch: 17, Train_loss: 0.3074213551837602
Epoch: 18, Train_loss: 0.304455556490575
Epoch: 19, Train_loss: 0.30284800522204025
Epoch: 20, Train_loss: 0.3005609646614175
Epoch: 21, Train_loss: 0.298796949206735
Epoch: 22, Train_loss: 0.29755487396905667
Epoch: 23, Train_loss: 0.29609404577

In [110]:
model_params

[Parameter containing:
 tensor([[-1.9122,  1.9629],
         [-0.0869,  0.0116],
         [-0.2007,  0.3290],
         [ 0.7662, -0.6207],
         [ 0.3159, -0.2075],
         [-0.3808,  0.0123],
         [-1.8392,  1.6507],
         [-0.1930, -0.1399],
         [-0.4842, -0.1984],
         [ 0.4997, -0.3252],
         [-0.1132,  0.2469],
         [ 0.1057,  0.1446],
         [-0.1633,  0.3999],
         [ 0.2557,  0.1728],
         [ 0.0118,  0.1075],
         [ 0.7299, -0.5774],
         [ 0.4315,  0.0640],
         [-0.5994,  0.1031],
         [-0.7230,  0.2252],
         [ 1.0827, -0.9988],
         [-0.8490,  0.6577],
         [ 1.4916, -1.4949],
         [ 1.8769, -1.6050],
         [ 0.0224, -0.1651],
         [-0.6884, -0.6645],
         [ 0.3962, -0.6373],
         [-0.9161,  0.8421],
         [-0.4830, -0.4958],
         [ 0.5741, -0.0684],
         [ 0.1493,  0.2979],
         [-0.2908,  0.2213],
         [-0.4026, -0.5232]], requires_grad=True),
 Parameter containing:
 ten

In [25]:
#w_1= 0.0467  w_2=0.0506

In [111]:
len(y_test)

80

In [112]:
y_pred=[]
for i in range(len(y_test)):
  _,pred=torch.max(model(x_test[i].unsqueeze(0)),dim=1)
  y_pred.append(pred.item())
len(y_pred)

80

In [113]:
y_pred=np.array(y_pred)
y_target=y_test.numpy()

y_pred.shape,y_target.shape

((80,), (80,))

In [114]:
acc = accuracy_score(y_target, y_pred)
prec = precision_score(y_target, y_pred)
f1 = f1_score(y_target, y_pred)
cm = confusion_matrix(y_target, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:\n", cm)

Accuracy: 0.9625
Precision: 0.9348
F1 Score: 0.9663
Confusion Matrix:
 [[34  3]
 [ 0 43]]


# Logistic Regression

In [115]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score

In [116]:
data_path="/content/data1.csv"

data_np=np.loadtxt(
    data_path, delimiter=",", dtype=np.float32,skiprows=1
)

data_np.shape

(400, 3)

In [117]:
x=data_np[:,:-1]
y=data_np[:,-1:]
y[y==-1],y[y==-1]=0,0

x.shape, y.shape

((400, 2), (400, 1))

In [118]:
x_train, y_train, x_test, y_test=x[:320], y[:320], x[320:], y[:320]
y_train,y_test=y_train.squeeze().astype(int), y_test.squeeze().astype(int)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((320, 2), (320,), (80, 2), (320,))

In [119]:
lg=LogisticRegression()
lg.fit(x_train, y_train)
y_pred=lg.predict(x_test)

In [120]:
y_pred

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1])

In [121]:
acc = accuracy_score(y_target, y_pred)
prec = precision_score(y_target, y_pred)
f1 = f1_score(y_target, y_pred)
cm = confusion_matrix(y_target, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:\n", cm)

Accuracy: 0.9875
Precision: 0.9773
F1 Score: 0.9885
Confusion Matrix:
 [[36  1]
 [ 0 43]]
